# 최단 경로 구하기
- 가중치가 없을 때의 최단 경로 : BF
- <b> 가중치가 있을 때의 최단 경로 : 다익스트라 알고리즘(Djikstra)</b>

## 다익스트라 알고리즘
- <b> 하나의 정점에서 다른 모든 정점으로 가는 최단 경로</b>를 알려줌
    - 음의 Edge를 포함하지 않아야 함
- <b> DP 문제 </b>이기도 하다. 어떤 최단 경로는 부분 최단 경로의 합이기 때문임

### 작동 과정
```
1. 출발 노드 설정
2. 출발 노드 기준 각 노드의 최소 비용 저장
3. 방문하지 않은 노드 중 가장 비용이 적은 노드를 저장
4. 해당 노드를 거쳐 특정 노드로 가는 경우를 고려하여 최소 비용을 갱신함
5. 3, 4를 반복함
```

# 1753번 : 최단 경로
- <b>방향 그래프</b>임!

In [27]:
# 데이터 초과

import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
V, E= map(int, input().split())
k = int(input())

# 가중치가 들어가니까 2차원 매트릭스는 필수?
matrix = [[int(1e9)] * (V+1) for _ in range(V+1)]
for _ in range(E):
    u, v, w = map(int, input().rstrip().split())
    matrix[u][v] = w   
matrix[k][k] = 0

# 최단경로 갱신 리스트 만들기
# 어떤 노드로 옮겼을 때 갱신해야 하는 값은 그 노드로 갔을 때의 값과 그 노드에서 다른 노드로 이동한 값 중 최솟값임
lst = matrix[k].copy()
for i in range(1, len(lst)): # 바운더리는 잘 모르겠지만?
    for j in range(1, len(lst)):
        lst[j] = min(lst[j], lst[i] + matrix[i][j])
        
        
for i in lst[1:]:
    print('INF') if i == int(1e9) else print(i)


0
2
3
7
INF


## 공간복잡도 개선
- 정답

In [16]:
import sys
import heapq
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
V, E= map(int, input().split())
k = int(input())

graph = [[] for _ in range(V+1)]

for _ in range(E):
    u, v, e = map(int, input().split())
    graph[u].append((v,e))
    
INF = int(1e9)
distance = [INF] * (V+1)

def dijkstra(start):
    
    heap = []
    heapq.heappush(heap, (0, start))
    distance[start] = 0
    
    while heap:

        dist, now_idx = heapq.heappop(heap)
        if distance[now_idx] < dist:
            continue
        
        for node in graph[now_idx]:
            
            cost = dist + node[1] # 지금 pop된 것까지의 거리 + 지금~다음 노드 까지의 거리
            
            if cost < distance[node[0]]:
                distance[node[0]] = cost
                heapq.heappush(heap, (cost, node[0]))
                
dijkstra(k)
for i in distance[1:]:
    if i == INF:
        print('INF')
    else:
        print(i)

0
2
3
7
INF


In [8]:
dijkstra(k)
for i in distance[1:]:
    if i == INF:
        print('INF')
    else:
        print(i)

0
2
3
7
INF


## 1504 : 특정한 최단 경로
- 1번 정점에서 N번 정점으로 이동하는 데,
    - 임의로 정해진 두 정점은 반드시 통과해야 한다  
      
      
~난 왜 문제를 갯수로 봤냐? 아타마가 오카시이인듯~
### 힐끔
- Dijkstra는 특정 노드 까지의 최단 거리를 구하는 알고리즘
- 그렇다면 각 노드마다 다른 노드까지의 최단 거리를 알 수 있다는 말인데, 꼭 지나야 하는 점이 2개 있음
    - 케이스가 2개로 나뉨 : `1 -> must1 -> must2 -> N` & `1 -> must2 -> must1 -> N`
- 함수 내에 최소 거리 리스트(`dp`)를 넣어서 해당 인덱스 값들만 더해주는데, 그 중 작은 값을 고르면 됨

In [19]:
import sys
import heapq
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
N, E= map(int, input().split())

graph = [[] for _ in range(N+1)]
for _ in range(E):
    a, b, c = map(int, input().split())
    graph[a].append((b,c))
    graph[b].append((a,c))
    
must1, must2 = map(int, input().split())

# 일단 기본 다익스트라를 구현
def dijkstra(start):
    heap = []
    distance = [float('inf')] * (N + 1)
    heapq.heappush(heap, (0, start))
    distance[start] = 0
    
    while heap:
        dist, now_idx = heapq.heappop(heap)
        
        if dist > distance[now_idx]:
            continue
            
        for node in graph[now_idx]:
            
            cost = dist + node[1]
            
            if cost < distance[node[0]]:
                distance[node[0]] = cost
                heapq.heappush(heap, (cost, node[0]))
                
    return distance

from_start = dijkstra(1)
from_must1 = dijkstra(must1)
from_must2 = dijkstra(must2)

ans = min(from_start[must1] + from_must1[must2] + from_must2[N], from_start[must2] + from_must2[must1] + from_must1[N])
print(ans if from_start[-1] < float('inf') else -1)